In [ ]:
import numpy as np
import qibo
from qibo import gates, hamiltonians, models, callbacks

# 为了结果可复现，可以设置一个随机种子
np.random.seed(42)

# --- 第 1 步：定义问题参数和哈密顿量 ---
# ------------------------------------------------

# 定义模型的物理参数
nqubits = 4      # 量子比特（自旋）的数量
h = 1.0          # 横向场强度 (h)
j_coupling = 1.0 # 相邻自旋相互作用强度 (J)

print(f"求解 {nqubits} 个量子比特的横向场伊辛模型")
print(f"参数: J = {j_coupling}, h = {h}\n")

# 使用 Qibo 的内置函数方便地创建 TFIM 哈密顿量
# H = -J * sum(Z_i Z_{i+1}) - h * sum(X_i)
# 注意：Qibo的TFIM默认J=1，所以我们在这里保持一致，但如果需要可以手动构建
hamiltonian = hamiltonians.TFIM(nqubits=nqubits, h=h)


# --- 第 2 步：构建参数化的量子线路 (Ansatz) ---
# ----------------------------------------------------

def create_ansatz(nqubits, nlayers):
    """
    创建一个硬件高效的 Ansatz (Hardware-Efficient Ansatz)
    由交替的 Ry 旋转层和 CNOT 纠缠层组成
    """
    # 创建一个包含 nqubits 的量子线路
    circuit = models.Circuit(nqubits)
    
    # 循环创建指定的层数
    for l in range(nlayers):
        # 参数化的单比特旋转层
        # 为每个量子比特添加一个 Ry 门，其角度是可训练的参数
        circuit.add(gates.RY(q, theta=0) for q in range(nqubits))
        
        # 纠缠层
        # 在相邻的量子比特之间添加 CNOT 门
        for q in range(0, nqubits - 1, 2):
            circuit.add(gates.CNOT(q, q + 1))
        
        for q in range(1, nqubits - 1, 2):
            circuit.add(gates.CNOT(q, q + 1))

    # 最后再加一层旋转门，以增加线路的表达能力
    circuit.add(gates.RY(q, theta=0) for q in range(nqubits))
    
    return circuit

# 设置 Ansatz 的深度（层数）
nlayers = 2
ansatz = create_ansatz(nqubits, nlayers)
print("构建的 Ansatz 结构:")
print(ansatz.draw())


# --- 第 3 & 4 步：执行 VQE 优化循环并找到最优解 ---
# ---------------------------------------------------------

# 初始化一组随机的初始参数 (角度)
# 参数的数量由 Ansatz 中可训练门的数量决定
initial_parameters = np.random.uniform(0, 2 * np.pi, ansatz.nparams)

# 使用 Qibo 的 VQE 模型
# 它将 Ansatz 和哈密顿量封装在一起
vqe = models.VQE(circuit=ansatz, hamiltonian=hamiltonian)

# 为了监控优化过程，我们可以使用一个回调函数
# 这个函数会在每次迭代时被调用，打印当前的能量值
energy_callback = callbacks.Energy(verbose=True)

# 运行 VQE 的核心 - 最小化过程
# 我们使用 'COBYLA' 这个经典的无梯度优化器
print("\n开始 VQE 优化 (使用 COBYLA 优化器)...")
best_energy, optimal_parameters = vqe.minimize(
    initial_params=initial_parameters,
    method='COBYLA',
    options={'maxiter': 150}, # 设置最大迭代次数
    callbacks=[energy_callback]
)


# --- 第 5 步：获得结果并与精确解对比 ---
# -----------------------------------------------

print(f"\n优化完成！")
print(f"VQE 找到的基态能量: {best_energy:.6f}")

# 对于小规模问题，我们可以计算精确的基态能量以作验证
# 1. 获取哈密顿量的矩阵形式
ham_matrix = hamiltonian.matrix
# 2. 对矩阵进行对角化，找到其最小的本征值
exact_eigenvalues, _ = np.linalg.eigh(ham_matrix)
exact_ground_energy = exact_eigenvalues[0]

print(f"精确对角化得到的基态能量: {exact_ground_energy:.6f}")

# 计算 VQE 结果与精确解的误差
error = abs((best_energy - exact_ground_energy) / exact_ground_energy)
print(f"相对误差: {error:.4%}\n")



实验目的
我们的核心目的是对Qiskit、PennyLane和Qibo这三个主流量子计算框架，在执行一个标准的、计算密集型的混合量子-经典任务（VQE求解TFIM基态）时，进行一次严谨、量化且公平的性能基准测试。
我们旨在超越简单的功能性对比，通过一个严格控制的实验环境，回答以下四个关键问题：
计算效率 (Efficiency): 在达到相同的求解精度目标下，哪个框架找到答案的速度最快（Time-to-Solution）？哪个框架所需的计算资源最少（Peak Memory Usage）？
可扩展性 (Scalability): 当问题规模（量子比特数）增加时，各个框架的计算效率（时间和内存）如何退化？哪个框架的性能增长曲线更为平缓，表现出更强的扩展潜力？
优化动力学 (Optimization Dynamics): 各个框架的VQE实现与经典优化器的交互效率如何？它们需要多少次成本函数求值才能收敛，其收敛过程是否平稳？
性能瓶颈诊断 (Bottleneck Diagnostics): 框架的性能瓶颈主要来源于底层的量子态向量模拟器（量子部分），还是来源于其外围的Python软件开销（经典部分）？
最终，我们的目标是生成一个多维度、数据驱动的性能画像，为量子算法研究者和开发者在选择最适合其特定计算需求（例如，快速原型验证、大规模数值模拟或教学）的软件框架时，提供客观、可复现且富有洞察力的决策依据。